# Imports

In [1]:
import torch
import torch.optim as optim
import numpy as np
import node_models
import loader
import training
import metrics
import autotune
import config

# GPU

In [2]:
device = torch.device(1)

# Data Loading

In [3]:
mnist_data_path = './train-images-idx3-ubyte'
mnist_label_path = './train-labels-idx1-ubyte'

In [4]:
#5-fold generation
mnist_data = loader.MNIST(mnist_data_path, mnist_label_path, 3)
mnist_splits = mnist_data.splits

In [5]:
print("MNIST STATS")
print("Number of splits:", len(mnist_splits))
print("Number of segments per split (train, val, test):", len(mnist_splits[0]))
print("Info per segment (data, labels):", len(mnist_splits[0][0]))
print("Size of segement (num examples):", len(mnist_splits[0][0][0]))

MNIST STATS
Number of splits: 3
Number of segments per split (train, val, test): 3
Info per segment (data, labels): 2
Size of segement (num examples): 40000


# Model & Optimizer

In [6]:
model = node_models.NeuralODE
optimizer = optim.Adam

# Training and Evaluation

## MNIST

### NODE

In [7]:
model_params = config.baseline_config_mnist_node['model']
lr, epochs, batch, workers = config.baseline_config_mnist_node['train']

In [8]:
node_mnist_trainer = training.Trainer(model, optimizer, mnist_data, device)

In [ ]:
node_mnist_trainer.train(model_params, lr, epochs, batch, num_workers=12, verbose=True, num_loss=3)

In [ ]:
node_mnist_trainer.test(model_params, 10, 8)

In [ ]:
 torch.cuda.empty_cache()

### ANODE

In [ ]:
model_params = config.baseline_config_mnist_anode['model']
lr, epochs, batch, workers = baseline_config.config_mnist_anode['train']

In [ ]:
anode_mnist_trainer = training.Trainer(model, optimizer, mnist_data, device)

In [ ]:
anode_mnist_trainer.train(model_params, lr, epochs, batch, num_workers=12, verbose=True, num_loss=3)

In [ ]:
anode_mnist_trainer.test(model_params, 10, 8)

# Plots

In [ ]:
node_mnist_trainer.val_metrics['legend'] = 'NODE'
anode_mnist_trainer.val_metrics['legend'] = 'ANODE'
metrics = [node_mnist_trainer.val_metrics, anode_mnist_trainer.val_metrics]

In [ ]:
plt = metrics.Plotter(metrics)

In [ ]:
plt.plotLoss("Optimized Model Validation Loss Comparisons")

In [ ]:
plt.plotAccuracy("Optimized Model Validation Accuracy Comparisons")